In [1]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('/Users/shuiguangshan/Mydocument/Grade 1 master/06 ML/02 CodeEveryday/01-regression/train.csv', encoding = 'big5')

In [2]:
data.head(20)

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [3]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

In [4]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

In [5]:
#每9个小时作为一个data预测第10个小时的pm2.5
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 *471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if (day == 19) & (hour > 14):
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:, day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
            y[month * 417 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9]


In [6]:
# 标准化
mean_x = np.mean(x, axis = 0) # 每个特征值取均值axis=0 列，-1为行，得到1 * （18*9）
std_x = np.std(x, axis = 0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [7]:
# 构造训练集和验证集
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]

### train 

In [8]:
# 线性回归 Y=WX
split_point = math.floor(len(x) * 0.8)
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 1000
iter_time = 100000
adagrad = np.zeros([dim, 1])
eps = 0.000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x,w) - y, 2)) / 471 / 12)
    if(t%10000==0):
        print(str(t) + ':' + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x,w) - y)
    adagrad += gradient**2
    w -= learning_rate * gradient / (adagrad + eps)
np.save('weight.npy', w)
w

0:24.990327215793748
10000:35.237590481740696
20000:34.03666223294716
30000:33.37756718875594
40000:32.93133877796954
50000:32.59881252068345
60000:32.33590876079819
70000:32.119335890335186
80000:31.935471509105124
90000:31.77577139596667


array([[ 5.78581196e-01],
       [-1.73580454e+00],
       [-2.12702172e+00],
       [-2.60741781e+00],
       [-3.02402723e+00],
       [-3.12636913e+00],
       [-2.89702343e+00],
       [-2.50525876e+00],
       [-1.98401836e+00],
       [-1.55755176e+00],
       [ 9.26562509e-02],
       [ 1.03553930e-01],
       [ 5.87174522e-02],
       [ 1.99399630e-02],
       [-1.68114466e-02],
       [-3.35418361e-02],
       [-6.08248365e-02],
       [-7.22803328e-02],
       [-7.58970751e-02],
       [-3.96371763e-02],
       [-9.95941381e-03],
       [ 1.63762960e-01],
       [ 2.86709548e-01],
       [ 3.41079049e-01],
       [ 2.89487139e-01],
       [ 1.94637683e-01],
       [ 1.13073029e-01],
       [ 1.88652266e-01],
       [-1.48734996e-01],
       [-1.66119484e-01],
       [-1.64843120e-01],
       [-1.26786940e-01],
       [-9.67113003e-02],
       [-1.02354712e-01],
       [-1.63420674e-01],
       [-2.36278946e-01],
       [-3.51924174e-01],
       [ 1.12163927e+00],
       [ 8.0

In [9]:
test_data = pd.read_csv('/Users/shuiguangshan/Mydocument/Grade 1 master/06 ML/02 CodeEveryday/01-regression/test.csv', header = None, encoding = 'big5')
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [10]:
test_data = test_data.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18 * 9], dtype=float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18 * (i + 1), :].reshape(1,-1)
## 标准化
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)

In [11]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)

In [12]:
import csv
with open('submit.csv', mode = 'w', newline='') as submit_flie:
    csv_writer = csv.writer(submit_flie)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 1.6484904573703811]
['id_1', 0.02185550361252797]
['id_2', 6.158177482134343]
['id_3', 0.6185765285276339]
['id_4', 1.2613267288018442]
['id_5', -0.45218449594607885]
['id_6', -0.24141514398062192]
['id_7', 0.5348689406747184]
['id_8', -0.5023680274342097]
['id_9', 3.3661853074361936]
['id_10', -0.6015058828064913]
['id_11', -6.704572493616354]
['id_12', 1.1441214621990947]
['id_13', 6.466370132931978]
['id_14', -1.1268781217516672]
['id_15', -1.9598871222215823]
['id_16', 2.976417548314364]
['id_17', 8.03323311356033]
['id_18', -5.344994378079468]
['id_19', -1.015669169069358]
['id_20', 1.6604562290037044]
['id_21', 3.9916881961710673]
['id_22', -2.213045719754961]
['id_23', -0.2805223485574395]
['id_24', -1.2057633435800605]
['id_25', 1.6088575829387044]
['id_26', -0.7137650237738078]
['id_27', 9.017064665480955]
['id_28', -1.821588083928844]
['id_29', 6.619170856563393]
['id_30', 3.348689910857498]
['id_31', 2.520657681532803]
['id_32', 0.16329164619425196]


In [ ]:
## 考虑正则化 正则化的lambda取多少合适，取validation的最小值的时候
## 考虑迭代次数
## 特征工程，变量之间的关系，加减乘除